In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
from sklearn.linear_model import LogisticRegression,RidgeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
%matplotlib inline

In [2]:
rawdf=pd.read_pickle('bigtrainingdataset.pickle')
df=rawdf.drop(['key','myteam','oppteam','gamedate','result'],axis=1)

In [3]:
df.head(2)

,myteam_prvwins,oppteam_prvwins,homegame,myteam_scores,oppteam_scores,myteam_trnovr,oppteam_trnovr,myteam_top,oppteam_top,myteam_pyds,oppteam_pyds,myteam_ryds,oppteam_ryds,myteam_penyds,oppteam_penyds,myteam_ptyds,oppteam_ptyds,myteam_curscore,oppteam_curscore
138,"[0, 0, 0, 0]","[0, 1, 1, 1]",False,"[17, 3, 14, 16]","[27, 19, 30, 23]","[2, 1, 2, 2]","[1, 0, 0, 3]","[26.5, 28.733333333333334, 35.083333333333336,...","[32.583333333333336, 32.71666666666667, 32.916...","[286, 258, 230, 150]","[408, 330, 246, 241]","[54, 7, 126, 182]","[46, 123, 146, 45]","[57, 51, 82, 67]","[53, 15, 40, 25]","[344, 331, 234, 259]","[140, 80, 52, 211]",19,23
140,"[0, 0, 1, 1]","[0, 1, 0, 0]",True,"[22, 20, 22, 13]","[13, 31, 28, 23]","[2, 4, 3, 3]","[4, 1, 1, 1]","[32.333333333333336, 25.683333333333334, 28.93...","[34.31666666666667, 22.183333333333334, 35.45,...","[204, 294, 160, 240]","[265, 273, 299, 399]","[30, 109, 90, 56]","[27, 18, 64, 58]","[22, 38, 30, 90]","[55, 101, 46, 55]","[197, 218, 199, 320]","[183, 192, 99, 137]",27,13


In [4]:
def GetAdjustedDF(df,adj_factor):
    
    def AdjustedStats (statlist,adj_factor) :
        if len(statlist)==3:
            return np.matmul(np.array(statlist),np.array([1+adj_factor,1,1-adj_factor]))
        elif len(statlist)==4:
            return np.matmul(np.array(statlist),np.array([1+adj_factor,1+adj_factor/2,1-adj_factor/2,1-adj_factor]))
    
    cols_to_transform=['myteam_prvwins', 'oppteam_prvwins','myteam_scores', 'oppteam_scores', 'myteam_trnovr', 'oppteam_trnovr',
       'myteam_top', 'oppteam_top', 'myteam_pyds', 'oppteam_pyds',
       'myteam_ryds', 'oppteam_ryds', 'myteam_penyds', 'oppteam_penyds',
       'myteam_ptyds', 'oppteam_ptyds']

    adjusteddf=df.copy()
    for col in cols_to_transform:
        adjusteddf[col]=df[col].apply(AdjustedStats,adj_factor=adj_factor)
        
    return adjusteddf

In [5]:
def Get_traintest(df):
    X,y=df.drop(['myteam_curscore','oppteam_curscore','myteam_penyds','oppteam_penyds',
             'myteam_prvwins','oppteam_scores',
#              'myteam_trnovr','oppteam_trnovr','myteam_top','oppteam_top'
            ],axis=1),df[['myteam_curscore','oppteam_curscore']]
    
    X=pd.DataFrame(StandardScaler().fit_transform(X),columns=X.columns)
    
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.35,shuffle=True)
    return (X_train,X_test,y_train,y_test)

In [6]:
X_train,X_test,y_train,y_test=Get_traintest(GetAdjustedDF(df,adj_factor=1))

/Users/abhijitshingote/miniconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype bool, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/abhijitshingote/miniconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype bool, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [7]:
from sklearn.linear_model import LinearRegression
regmod=LinearRegression()
regmod.fit(X_train,y_train)
regmod.predict(X_train)

/Users/abhijitshingote/miniconda3/lib/python3.6/site-packages/sklearn/linear_model/base.py:485: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  linalg.lstsq(X, y)


array([[20.27054895, 20.94544383],
       [23.65382937, 23.35587179],
       [18.52284581, 25.90826208],
       ...,
       [24.53815165, 18.57424098],
       [22.293438  , 26.64042465],
       [17.52083044, 25.11723299]])

In [8]:
#  Train Test Ready above... Training to follow

In [9]:
mydictofmodels=[
    {
        'estimator':LogisticRegression(),
        'name':'LogisticRegression',
        'parameters':{
            'solver':['lbfgs','liblinear'],
            'C':[1.0,0.8,0.5,0.3]
        }
    },
        {
        'estimator':RidgeClassifier(),
        'name':'Ridge Classifier',
        'parameters':{
            'alpha':[1.0,2.0,10.0,0.5],
            'solver' : ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag']
        }
    },
        {
        'estimator':GaussianNB(),
        'name':'Gaussian Naive Bayes',
        'parameters':{
#             'solver':['lbfgs','linear']
        }
    },
            {
        'estimator':DecisionTreeClassifier(),
        'name':'Decision Tree',
        'parameters':{
            'min_samples_split':[2,5,10,20,40],
            'min_samples_leaf' :[20,40,60,100],
            'max_depth':[3,5,10,25]
        }
    },
            {
        'estimator':RandomForestClassifier(),
        'name':'Random Forest',
        'parameters':{
            'min_samples_split':[2,5,10,20,40],
            'min_samples_leaf' :[20,40,60,100],
            'n_estimators':[100]
        }
    },
    {
        'estimator':SVC(),
        'name':'SVC',
        'parameters':{
            'gamma':['auto','scale']
        }
    }
    
]

In [10]:
modelnames=[]
modelscores=[]
modelbestparams=[]
adj_factor_list=[]

for adj_factor in [0,0.1,0.2,0.3,0.4,0.5]:
    adjdf=GetAdjustedDF(df=df,adj_factor=adj_factor)
    X_train,X_test,y_train,y_test=Get_traintest(adjdf)
    for model in mydictofmodels:
        gsmodel=GridSearchCV(model['estimator'],param_grid=model['parameters'],cv=5,verbose=True)
        gsmodel.fit(X_train,np.array(y_train).ravel())
        modelnames.append(model['name'])
        modelscores.append(gsmodel.score(X_test,y_test))
        modelbestparams.append(gsmodel.best_params_)
        adj_factor_list.append(adj_factor)


/Users/abhijitshingote/miniconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype bool, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/abhijitshingote/miniconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype bool, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


ValueError: Found input variables with inconsistent numbers of samples: [878, 1756]

In [ ]:
resultsdf=pd.DataFrame({'modelname':modelnames,
              'scores':modelscores,
              'bestparams':modelbestparams,
              'adj_factor':adj_factor_list})
resultsdf.sort_values(['scores'],ascending=False).head(10)

In [ ]:
# mlpmodel=MLPClassifier(solver='adam', alpha=1e-5,
#                        hidden_layer_sizes=(5,10,5), early_stopping=False,random_state=1,max_iter=1000,verbose=True,learning_rate_init=0.001)
# mlpmodel.fit(X_train,np.array(y_train).ravel())
# mlpmodel.score(X_train,y_train),mlpmodel.score(X_test,y_test)